In [64]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage.filters import correlate 
import matplotlib.pyplot as plt
import matplotlib.image as pltimg 
from scipy.spatial.distance import cdist
from scipy.spatial import distance



In [42]:
def load_gray_to_double(path):
    '''
    :param path: path to load image
    :return: grayscaled and float image
    '''
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # transmit the image to float
    img = img.astype(np.float64)/255.0

    print(img.shape)
    return img

def load_rgb(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    return img


def show_img(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    
    
def normalize(img):
    ''' Function to normalize an input array to 0-1 '''
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)

In [43]:
"""
Harris Corner Detector
Usage: Call the function harris(filename) for corner detection
Reference   (Code adapted from):
             http://www.kaij.org/blog/?p=89
             Kai Jiang - Harris Corner Detector in Python
             
"""
from pylab import *
from scipy import signal
from scipy import *
import numpy as np
from PIL import Image

def harris(filename, min_distance = 10, threshold = 0.1):
    """
    filename: Path of image file
    threshold: (optional)Threshold for corner detection
    min_distance : (optional)Minimum number of pixels separating 
     corners and image boundary
    """
    im = np.array(Image.open(filename).convert("L"))
    harrisim = compute_harris_response(im)
    filtered_coords = get_harris_points(harrisim, min_distance, threshold)
    plot_harris_points(im, filtered_coords)
    return filtered_coords

def gauss_derivative_kernels(size, sizey=None):
    """ returns x and y derivatives of a 2D 
        gauss kernel array for convolutions """
    size = int(size)
    if not sizey:
        sizey = size
    else:
        sizey = int(sizey)
    y, x = mgrid[-size:size+1, -sizey:sizey+1]
    #x and y derivatives of a 2D gaussian with standard dev half of size
    # (ignore scale factor)
    gx = - x * exp(-(x**2/float((0.5*size)**2)+y**2/float((0.5*sizey)**2))) 
    gy = - y * exp(-(x**2/float((0.5*size)**2)+y**2/float((0.5*sizey)**2))) 
    return gx,gy

def gauss_kernel(size, sizey = None):
    """ Returns a normalized 2D gauss kernel array for convolutions """
    size = int(size)
    if not sizey:
        sizey = size
    else:
        sizey = int(sizey)
    x, y = mgrid[-size:size+1, -sizey:sizey+1]
    g = exp(-(x**2/float(size)+y**2/float(sizey)))
    return g / g.sum()

def compute_harris_response(im):
    """ compute the Harris corner detector response function 
        for each pixel in the image"""
    #derivatives
    gx,gy = gauss_derivative_kernels(3)
    imx = signal.convolve(im,gx, mode='same')
    imy = signal.convolve(im,gy, mode='same')
    #kernel for blurring
    gauss = gauss_kernel(3)
    #compute components of the structure tensor
    Wxx = signal.convolve(imx*imx,gauss, mode='same')
    Wxy = signal.convolve(imx*imy,gauss, mode='same')
    Wyy = signal.convolve(imy*imy,gauss, mode='same')   
    #determinant and trace
    Wdet = Wxx*Wyy - Wxy**2
    Wtr = Wxx + Wyy   
    return Wdet / Wtr

def get_harris_points(harrisim, min_distance=10, threshold=0.1):
    """ return corners from a Harris response image
        min_distance is the minimum nbr of pixels separating 
        corners and image boundary"""
    #find top corner candidates above a threshold
    corner_threshold = max(harrisim.ravel()) * threshold
    harrisim_t = (harrisim > corner_threshold) * 1    
    #get coordinates of candidates
    candidates = harrisim_t.nonzero()
    coords = [ (candidates[0][c],candidates[1][c]) for c in range(len(candidates[0]))]
    #...and their values
    candidate_values = [harrisim[c[0]][c[1]] for c in coords]    
    #sort candidates
    index = argsort(candidate_values)   
    #store allowed point locations in array
    allowed_locations = zeros(harrisim.shape)
    allowed_locations[min_distance:-min_distance,min_distance:-min_distance] = 1   
    #select the best points taking min_distance into account
    filtered_coords = []
    for i in index:
        if allowed_locations[coords[i][0]][coords[i][1]] == 1:
            filtered_coords.append(coords[i])
            allowed_locations[(coords[i][0]-min_distance):(coords[i][0]+min_distance),
                (coords[i][1]-min_distance):(coords[i][1]+min_distance)] = 0               
    return filtered_coords

def plot_harris_points(image, filtered_coords):
    """ plots corners found in image"""
    figure()
    gray()
    imshow(image)
    plot([p[1] for p in filtered_coords],[p[0] for p in filtered_coords],'r*')
    axis('off')
    show()

#harris('./CS543_ECE549 Assignment 3_files/sample_panorama.JPG')

In [44]:
"""
    Extract  descriptor
    returns a descriptor of size numoffeature * (neighbor+1)*(neighbor+1)
"""
def descriptor_extract(img, neighborhoods, feature_points):
    num_of_features_ponts = len(feature_points)
    print(num_of_features_ponts)
    descriptors = np.zeros((num_of_features_ponts, (2*neighborhoods+1)**2))
    tmp_img = img.copy()
    
    for i in range(num_of_features_ponts):
        # obtain img slices
        # !! Remember to +1 for slice right part! 
        tmp_img = img[feature_points[i][0]-neighborhoods:
                      feature_points[i][0]+neighborhoods+1,
                  feature_points[i][1]-neighborhoods:
                  feature_points[i][1]+neighborhoods+1]
        descriptors[i, :] = tmp_img.reshape((1, (2*neighborhoods+1)**2))
        
    return descriptors

In [57]:
np.random.choice([1,2,3], 2)

array([3, 3])

In [123]:
def dist2(x,y):   
    return np.sqrt(np.sum((x-y)**2))

# RANSAC
# matches: numberof features * 2 coords
# ((points_left_x,points_right_y,points_right_x,points_right_y))
def RANSAC(matches):
    num_Of_selected = len(matches)
    print("num of matches",num_Of_selected)
    # start number for each iteration
    sample_numbers = 4
    threshold = 5
    iterations = 200
    cnt = 1
    
    while cnt < iterations: 
        # get 4 random samples from matches
        if sample_numbers == 4:
            # get sample from matches
            inliers = np.random.choice(num_Of_selected, sample_numbers)
            

       
        Matrix = np.array([])
        # for each sample:
        for j in range(sample_numbers):
            current_match = matches[inliers[j]]
            #left feature point
            xt = np.array([current_match[1], current_match[0], 1])
            new_row = np.append(np.append(0*xt, xt), -current_match[2]*xt)
            # new_row = np.array([0*xt,xt,-current_match[2]*xt])
            new_row2 =np.append(np.append(xt, 0*xt), -current_match[3]*xt)
            if len(Matrix) == 0:
                Matrix = new_row
            else:
                Matrix = np.vstack([Matrix, new_row])
            Matrix = np.vstack([Matrix, new_row2])
        
        #get last line 
        _,_,VMatrix = np.linalg.svd(Matrix)
        HMatrix = VMatrix[:, -1]
        H_reshape = np.reshape(HMatrix, (3, 3))
        
        # print(H_reshape)
        inliers_number = 0
        inliers = []
        residual = []
        
        for i in range(len(matches)):
            #X is 3*3?
            X = np.matmul(np.transpose(H_reshape), np.array([matches[i][1],
                                                    matches[i][0], 1])
                          )
            x = X[0]/X[2]
            y = X[1]/X[2]
            sample_point = np.array([x,y])
            match_point = np.array(matches[i][3], matches[i][2])
            tmpDis = dist2(sample_point, match_point)
            print(tmpDis)
            if tmpDis < threshold:
                inliers += [i]
                residual += [tmpDis]
                inliers_number += 1
            
        if inliers_number < 15:
            sample_numbers = 4
            print("inliernumber:",inliers_number)
        else:
           sample_numbers = inliers_number
           cnt += 1
           print("got one")
            
    mean_of_residual = np.mean(residual)
    print(inliers_number)    
    return inliers, inliers_number, mean_of_residual, H_reshape



In [114]:
def show_features(img_l,img_r,matches,final_matches):
    implot_l = plt.imshow(img_l)
    print(len(matches))
    print(len(final_matches))
    plt.scatter(x=matches[:, 1], y=matches[:, 0], c='r', s=10)
    plt.scatter(x=final_matches[:, 1], y=final_matches[:, 0], c='g', s=10)
    plt.show()
    
    implot_r = plt.imshow(img_r)
    plt.scatter(x=matches[:, 3], y=matches[:, 2], c='r', s=10)
    plt.scatter(x=final_matches[:, 3], y=final_matches[:, 2], c='g', s=10)
    plt.show()


In [88]:
def gauss_img(img, sigma):
    print(img.shape)
    tmp_img = img
    kernel = gauss_kernel(2*sigma+1)
    print(img.ndim)
    if img.ndim == 3:
        for i in range(img.ndim):
            tmp_img[:, :, i] = correlate(tmp_img[:, :, i], kernel)
    else:
        tmp_img = correlate(tmp_img, kernel)
    return tmp_img
        

In [89]:
img_1_left_path = "data\\part1\\left.jpg"
img_1_right_path = "data\\part1\\right.jpg"

In [71]:
# load image
img1_left_rgb = load_rgb(img_1_left_path)
img1_right_rgb = load_rgb(img_1_right_path)
img1_left_gray = load_gray_to_double(img_1_left_path)
img1_right_gray = load_gray_to_double(img_1_right_path)
# img1_left = load_image(img_1_left_path)
# img1_right = load_image(img_2_right_path)
# show_img(img1_left)
# show_img(img1_right)
neighborhoods = 5

# harris Cornor detect feature points 
feature_points_left = harris(img_1_left_path, min_distance=2*neighborhoods)
feature_points_right = harris(img_1_right_path, min_distance=2*neighborhoods)
# print(feature_points_left)




(398, 800)
(398, 800)


C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: divide by zero encountered in true_divide


In [72]:
"""
Extract local neighborhoods around every keypoint in both images
and form descriptors simply by "flattening" the pixel values in each neighborhood 
to one-dimensional vectors.
"""
descriptor_left = descriptor_extract(img1_left_gray, 
                                      neighborhoods, feature_points_left)
descriptor_right = descriptor_extract(img1_right_gray, 
                                       neighborhoods, feature_points_right)


598
376


In [124]:
"""
Compute distances between every descriptor in one image and every descriptor in the other image. 
In Python, you can use scipy.spatial.distance.cdist(X,Y,'sqeuclidean') 
for fast computation of Euclidean distance. 
"""
# dim of distance: num_Of_left * num_of_right
distance = cdist(descriptor_left, descriptor_right, 'sqeuclidean')
"""
Select putative matches based on the matrix of pairwise descriptor distances obtained above. 
You can select all pairs whose descriptor distances are below a specified threshold, 
or select the top few hundred descriptor pairs with the smallest pairwise distances.
"""

# num of points to extract 
total_select_numbers = min(300, len(feature_points_right), len(feature_points_left))
select_match = []
INF = 1111111
for i in range(total_select_numbers):
    # find matrix minimum index
    ri, ci = np.unravel_index(distance.argmin(), distance.shape)
    select_match += [(feature_points_left[ri][0],feature_points_left[ri][1],
                      feature_points_right[ci][0],feature_points_right[ci][1])]
    # set to INF after used
    distance[ri, :] = INF
    distance[:, ci] = INF
    
"""
Implement RANSAC to estimate a homography mapping one image onto the other. 
Report the number of inliers and the average residual for the inliers 
(squared distance between the point coordinates in one image and
 the transformed coordinates of the matching point in the other image). 
Also, display the locations of inlier matches in both images.
"""
[inliers, inliers_number, mean_of_residual, H] = RANSAC(select_match)
final_Match = []
print("final match number is",inliers_number)
for i in range(inliers_number):
    final_Match += [select_match[inliers[i]]]

# show features
show_features(img1_left_rgb,img1_right_rgb,
              select_match,  final_Match)

 

    


num of matches 300
381.8094783138191
347.62614709045056
916.5477731745094
281.2592476023283
189.58473971608547
1099.8143821514589
854.6733119036182
651.7791225406845
29.858709381815792
207.97243146016913
505.0046366200717
299.9260227813192
1015.2788206605954
112.0748799679667
1077.18344565284
1085.1551699140775
372.9397153652535
635.3670555575741
1071.677420691808
360.5335231824474
948.8003931439271
815.7929541472474
202.7319949468981
151.50238666391473
806.7079847111738
1021.9561503555791
251.83165060861694
231.9542379465803
539.8658308539349
464.21595575157414
418.5155277844947
672.1134542174585
1079.6539570895368
1041.3536423407577
981.125917614701
1039.1956381060843
624.0013484934583
1062.8373838939856
855.2238081641614
845.9519317217331
1060.1925184045526
369.2837619490358
570.4392612918806
1068.248389413252
835.1012862044446
404.2875934559316
749.9436025849294
155.4278744226578
86.02874688680022
605.6004414256784
718.7401591570367
423.88470864165356
1086.4997278994474
608.0182766

88.14950813735837
299.00013647336857
147.50877453826712
964.2913509132603
309.5112038814823
283.6870474936359
943.2838386249638
456.65785486303264
409.13568314271697
83.18268739572935
225.27998718479415
1003.9192456128773
47.7242213850345
711.7286674334317
660.6231771655592
673.5994091667445
134.67644976840353
66.19024687883058
540.4574534111804
391.7363358181747
975.6874586131165
211.7104967962397
156.06226218680501
752.6371492314232
129.97679150175725
169.50772718955508
443.1350104722524
501.0857942835053
65.42748512037782
306.97300265813055
370.5253984129286
293.3883973306356
321.00708137555455
715.1269753913002
140.98073035949474
84.59616306322899
494.6059050725603
135.40778089420436
1023.326224008005
989.3547113944494
922.2489996569548
1111.784938652671
inliernumber: 0
396.4813784454504
355.4277016642586
929.7502333996097
297.3798628080506
203.52618866086797
1118.8688876135027
883.378021322161
667.7319961944282
44.88614189812052
221.8565827809895
518.317680292276
313.4675361445765


731.0531324652055
439.38103438562547
1105.3070697853298
622.1331290869499
394.6175998717046
794.855824499741
236.6859404555794
464.1276809018987
990.7273506699056
398.99677417569944
1068.4986772447605
1109.3766788983496
719.0598005422869
799.6871215098457
315.73449119054743
678.7853991444832
928.8870339200546
547.1329930095645
651.7399828113356
184.25063465015558
491.8406595549932
344.40964046304526
499.07815858030756
684.3390118855787
579.864880862349
145.07707388169243
602.3458186747148
1126.8727282401271
198.4276950598603
899.8880851706067
1028.7781663254082
1103.7753229425525
1073.266402198176
785.5750972535508
420.97617871436546
1054.4540295302618
689.1714303890651
136.10939791907833
203.9411767686871
566.1659483926634
665.462597490116
242.0060574817212
289.2649337775735
770.5056060987057
996.4541226247236
917.9043951086737
1069.2223328764135
701.4132207793305
1094.6259304014723
319.7612806957097
328.1331755189706
1120.5572962316269
285.58441625670866
833.7686100301758
953.045256

47.98921826610876
70.61556835131796
303.9592651246729
308.2027947362439
726.8085217852795
946.0138917394893
381.74214014118627
91.82922499199873
930.4559365688319
46.574041884084934
1005.4111460955614
325.17197383032925
1111.4767156924593
387.3983384840931
97.48271604086489
176.67956691235244
982.7820781480673
664.5853307602936
296.8859286543944
863.9893600538174
848.4324440367685
408.61309596200095
309.61835273508404
1107.2250976168557
764.9948555500116
195.0667684337911
752.266992974948
960.1565725927196
234.66206547910073
927.6230776278913
414.2670530397888
475.0782476712084
479.3220197203398
1108.6476365621256
591.0460443291277
899.3443372827181
462.3506665203239
875.3037917147017
1036.524573580294
424.16927358796056
460.93289834090945
848.4323402030252
214.86223231271464
384.5705987728745
721.153594680198
100.31492165861776
70.61480041103952
948.8394401141732
944.5952495521857
330.8269905744344
439.7170567332094
521.7505541051585
415.67714542134
469.42312322656767
789.035261325446


459.894784453158
56.097116471126384
382.50781548833453
740.0875510643667
404.65545800097965
410.1476805113916
963.6075731643979
413.6198003009266
338.10354477765316
1016.9595652142676
177.2308176942775
1051.0075466885753
420.07713906389824
1113.784561579933
442.87756465636187
991.4113348326322
406.14040425116383
898.0748599473045
431.3893335552966
176.77181149939486
432.7448391290445
105.117533993379
250.41042318754117
772.392598809917
125.97470521281114
34.12569922578687
249.50458909333042
1005.4998195550849
363.4479609331322
624.9095269138487
26.07592712803169
1076.3031315173098
1112.0216357323573
30.001843825726095
909.3036559622443
1103.2371600160807
437.19672281507314
431.3602728226625
827.93291227565
985.9803026734522
462.7424451362608
352.4301189084599
834.44535203196
741.1513913367825
80.84276643373664
317.39036879919615
149.7756510401424
959.5003659782392
304.5285203350607
281.41251259136203
937.7523660289006
452.5982689371082
401.1729227047763
78.03704284018912
220.717824987

...

612.3959457047761
48.1230697009218
70.75184841856236
304.0991710631927
308.33942081292696
726.9495917381045
946.1490060203155
381.8790949977741
91.96435606565966
930.5955710639288
46.709577698968246
1005.5457589642587
325.31179008111116
1111.6125767824913
387.5369869386287
97.6256088576841
176.82074396139384
982.9205781712678
664.7205522641901
297.03034444164444
864.1246105423112
848.5687240510623
408.74780216871886
309.75157545270577
1107.37171142346
765.1291895212279
195.20184532717172
752.4014920240394
960.2895078689165
234.80152549489011
927.7722567320914
414.40893608053176
475.2199283223813
479.46132451683013
1108.7833537605538
591.1811304512994
899.4793433529926
462.4918023665461
875.4375677354333
1036.6570362379343
424.30439019919424
461.07528963792487
848.5671364681269
215.00534399838855
384.7073126012398
721.2878702859579
100.44747635019912
70.75315237909125
948.9820568441634
944.7410605493995
330.97178905379735
439.8708836884467
521.8839919345997
415.8275600075416
469.5608700


339.11977187356405
699.883484751952
987.211224142521
588.1065252897563
53.56215491492036
942.2793264399073
332.1455951801186
1081.158478419353
296.85447671592465
727.1667908564036
616.3839942295623
52.590711898140064
75.03272591074467
308.14077884004996
312.5443211362573
730.9116079330115
950.4781484126429
386.1024818177128
96.32798558947847
934.6496985456865
51.01671429713742
1009.9212296504899
329.30865412785994
1115.9046490338653
391.65282408099256
101.48169128544562
180.790337841584
987.0229911975354
669.0397019991351
300.78779578847167
868.4335529296372
852.7896611760407
413.12931109061907
314.1965665520167
1110.757155256033
769.5134763738432
199.56352879756335
756.7887699997818
964.7403240653676
238.78327146432142
931.30236241769
418.33006037501264
479.1461838051419
483.54282386683366
1113.0063572061777
595.4826707055321
903.7531002230255
466.4596131525294
879.8704680879391
1041.1809241220794
428.6550244629373
464.73556936720416
852.8115960886972
218.84167693938275
388.939821651

1121.5796322832168
382.3622919278827
264.8216688129449
570.1643842047076
504.10554243120447
489.2314455943361
320.27563566081164
494.14227868153995
522.8525265545622
1002.877862919917
983.2323395249975
335.52537929239253
696.2788458589814
983.9980097725488
584.4976999080054
49.92939187743705
939.1709666263658
328.53462937998995
1077.8168359157787
293.2518798753328
729.2368452781416
612.7758047934142
51.93720605562902
71.68232782243845
304.5320597994134
309.06155355875865
727.3062067781105
947.4790713414102
382.6592872002936
93.48188960647815
931.0471356639657
47.714120690567334
1007.4052392689598
325.69794064105406
1112.6923138309887
388.0734159061652
97.9000887872282
177.17885173356385
983.4389922517638
665.9668921336855
297.2591544974125
865.3028373404551
849.3464204766464
410.6699227930906
313.7400529607105
1107.4937997636762
767.1244490841501
196.7924546006312
754.4548978682134
965.0142610835923
235.17227534186105
927.9226678256133
414.732083032487
475.5472256390135
479.94695278729


780.4696236360203
633.3964418974174
675.8247663886943
33.762986063363996
875.2110092730835
653.1966380173467
1109.97780361257
381.65334777145387
667.3439057542034
1107.1508800252216
1115.6473609330374
381.6546232401505
264.2678058225551
549.9632224501999
503.28118775445205
484.90763078093164
319.4333457405006
493.3796388198887
504.70818650846894
1002.4608202096255
982.686378562926
334.9766041135948
695.6087215780832
982.7051162660522
583.8831028583387
49.31126706757847
938.7935503985464
327.91074575572185
1077.4132188388685
292.5590807741799
722.4967129378932
612.1682932000567
47.91609767982129
70.53645858039711
303.8709709808824
308.12177525382486
726.7168716310471
945.9370370805325
381.66122615121134
91.75295819683224
930.368279494066
46.49615878775574
1005.3356974600046
325.08255480584535
1111.3983767892416
387.31322031052395
97.38580913730283
176.58795798523303
982.6967937794742
664.5081636550539
296.78418781880566
863.9119580661709
848.3521492789972
408.53766204715583
309.5455205

KeyboardInterrupt: 